# SQL Postgres n SQLITE

* KPIs in sql

somes refs:

    * customer retention: https://blog.statsbot.co/customer-retention-analysis-93af9daee46b
    * funnel analysis: https://blog.statsbot.co/sql-queries-for-funnel-analysis-35d5e456371d
    
        How many subscribe -> how many did audiction -> How many order an report
    
    * GAMES KPIs
    
        * https://applift.com/blog/user-retention
        * https://medium.com/n3twork/game-analytics-overview-ea8f53ffdff1
        * https://www.periscopedata.com/blog/how-to-calculate-cohort-retention-in-sql #A

* windows functions
* joins
* Filters 
* Transactions #A very important
* jaccard and trigrmas (at least postgres support that)

* How to insert csv file in sqlite3 in summarize data without worry about memory issues


In [4]:
import numpy as np
import pandas as pd

from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sqlalchemy import create_engine
import datetime as dt


## SQLITE

* Search how to run sql code in jupyter notebook cells
* terminal common command
* common SQL queries
    * joins
    * cross join (cartesian product)
* windowing functions (time series) GOOD explanation

### Code to process and add data to sqlite file


If the data does not fit in memmory and it is not big data, an alternative is to load the data in sqlite and use sqlite to summarize the data. In the case you need median and qunatiles you can take a large sample that fits in memmory and use python to compute that, so you will have a good estimation of these statistics.

refs: https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/

In [1]:
!head -n 1 data/logs.log

In [5]:
disk_engine = create_engine('sqlite:///logs.db') 

chunksize = 3
j = 0
start = dt.datetime.now()

file_name = 'data/logs.log'

columns =  [ 'date', 'time', 
            'site', 'computer_name', 'server_ip', 
            'request_type', 'uri_stem', 'uri_query', 'port', 
            'user_name', 'remote_ip', 'cs_version', 'browser', 'referer', 'host',
            'http_code', 'sub_status','win32_status',
            # https://support.symantec.com/en_US/article.TECH243050.html
            # cs-bytes: Number of bytes sent from client to server
            # sc-bytes: Number of bytes sent from server to client
            'sc_bytes','cs_bytes',
            'loading_time' ]

columns_to_drop = ['computer_name', 'uri_query', 'user_name', 'cs_version', 
                   'host', 
                   'sub_status','win32_status' ]

for df in pd.read_csv(file_name, sep=' ',index_col=False, header=None, chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    print(df.shape)
    df.columns = columns
     
    # drop columns
    for c in columns_to_drop:
        df = df.drop(c, axis=1)
  
    # Do MORE processing in the columns

    # Insert data
    df.to_sql('log', disk_engine, if_exists='append')
    
    j+=1
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

    #assert j <= 3, "Break time"

(3, 21)
0 seconds: completed 3 rows
(3, 21)
0 seconds: completed 6 rows
(3, 21)
0 seconds: completed 9 rows
(2, 21)
0 seconds: completed 12 rows


In [8]:
!ls
!sqlite3 logs.db 'SELECT * FROM  log LIMIT 3;'

0|2018-10-01|07:00:16|W3SVC1574643156|10.0.1.50|GET|/castingworkbook/directors/director_home.asp|80|10.0.1.50|Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_12_6)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/11.1.2+Safari/605.1.15|http://www.castingworkbook.com/castingworkbook/directors/dmenu2.asp|200|16171|1539|375
1|2018-10-01|07:00:16|W3SVC1574643156|10.0.1.50|GET|/castingworkbook/newFeatures.asp|80|10.0.1.50|Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_12_6)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/11.1.2+Safari/605.1.15|http://www.castingworkbook.com/castingworkbook/directors/director_home.asp|200|189|1545|15
2|2018-10-01|07:00:18|W3SVC1574643156|10.0.1.50|GET|/castingworkbook/agents/shows.asp|80|10.0.1.50|Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_13_6)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/12.0+Safari/605.1.15|http://www.castingworkbook.com/castingworkbook/agents/shows.asp?type=invitation|200|88306|1095|312


## Postgresq


Ref: https://gitlab.com/leandroohf/postgres_train (my repo in GitLab)

* Search how to run sql code in jupyter notebook cells
* terminal common command
* common SQL queries
    * joins
    * cross join (cartesian product)
    * find the name of the special join that allows you in  a subquery use keyword defined outside
* KPIS (since is more likelly to use here and postgresql is more popular)
* windowing functions (time series) GOOD explanation


### Start postgresql server

* https://www.digitalocean.com/community/tutorials/how-to-install-and-use-postgresql-on-ubuntu-18-04
* https://www.digitalocean.com/community/tutorials/how-to-install-postgresql-on-ubuntu-20-04-quickstart

```sh

# Install
sudo apt update
sudo apt install postgresql postgresql-contrib


# double check installation
sudo -u postgres psql -c "SELECT version();"


# switch to posthgresl user
sudo -i -u postgres

# or 
sudo -u postgres psql

## Inside psql prompt
# create a new db
CREATE DATABASE time_series;

\list

## connect to time_series db using client like dbeaver

# change default password for user postgres in psql shell
# https://enterprise.arcgis.com/en/server/10.3/cloud/amazon/change-default-database-passwords-on-linux.htm
 \password postgres

```

### Data

#### Upload data to a db

ref:
* https://www.cybertec-postgresql.com/en/postgresql-detecting-periods-of-activity-in-a-timeseries/


* From csv file


```sh
psql -U postgres -d bike_data -h localhost -c "\COPY bikes FROM bikeshare.csv CSV"

```

* From SQL 

```sql

-- https://www.cybertec-postgresql.com/en/postgresql-detecting-periods-of-activity-in-a-timeseries/
-- XXX: NOT WORKING. NEED TO DISCOVER WHY?
CREATE TABLE t_series (t date, data int);
 
COPY t_series FROM stdin DELIMITER ';';
2018-03-01;12
2018-03-02;43
2018-03-03;9
2018-03-04;13
2018-03-09;23
2018-03-10;26
2018-03-11;28
2018-03-14;21
2018-03-15;15
\.

```

* From dataframe


```python

df.to_sql()

```


#### Read with pandas for one time analysis

```python

from sqlalchemy import create_engine

user = 'x'
pwd = 'x'
host = 'xxxx'
db = 'db'

engine = create_engine(f'postgresql://{user}:{pwd}@d{host}:5432/{db}')

query = """
select *
from table_eith_data t 
"""

# XXX: BEcareful. This line wil replace any existing table
demo_df = pd.read_sql_query(query, engine)
```

#### Read from db by chunks (For large data sets)

In [ ]:
#TODO:

### Common operation


#### Tables

* Create Table  

```sql

-- logins table  
CREATE TABLE IF NOT EXISTS users.logins (
ID id SERIAL     --- auto increment. sqlite the keywords is "autoincrement"
"date" DATE UNIQUE,
user_type INT,
n_logins INT,
UNIQUE ("date")  -- constraint: date should be unique
PRIMARY KEY ID
);

```

* Clear data in table 


```sql

DELETE FROM medias;


-- BETTER
DELETE FROM medias 
WHERE media_id = '1234';

```


* Delete Table

```sql

DROP TABLE medias.growth;

DROP DATABASE "voicematchseedtest";
```


* Contraints

https://pganalyze.com/docs/log-insights/app-errors/U125  
https://www.postgresqltutorial.com/postgresql-unique-constraint/  
https://www.postgresql.org/docs/9.4/ddl-constraints.html  

So you can use things like ON CONFLICTS DO SOMETHING


```sql

CREATE TABLE IF NOT EXISTS users.logins (
"date" DATE,
user_type INT,
n_logins INT,
UNIQUE ("date", user_type)  -- constraint unique pair 
);


-- You need to get the name of the constraint on dbeaver client
INSERT INTO users.logins ("date", user_type, n_logins) 
VALUES ('2020-08-03', 8, 2)
ON CONFLICT ON CONSTRAINT logins_date_user_type_key   -- constraint name. See dbeaver
DO nothing;

```


#### CTE

Helps a lot with analytcs
1. It is easy to debug and understand
1. We can build the cte tables by parts and check the results
1. The query is more readbale and easy to add commentaries 

Example with muliple CTEs

```sql
WITH Active_accounts_cte (gender, cityid, city, yob, clientid, serviceid) -- complex table with all active account candidates
        AS
        (
            SELECT c.gender, c.residence, lc.description,
                CONVERT(varchar(4),c.birthday , 120) AS yob,
                clientid,
                servicerid AS serviceid
            FROM [clients].[dbo].[tblclients] c
            LEFT JOIN [casting]..lstCities lc ON lc.cityID = c.residence
            WHERE c.[status] > 0  -- complex criteria 
                AND servicerid <> 94 -- exclude phanthom service
        ),
        Roster_cte (clientid)  -- complex table with all actors that is a member of a roster
        AS
        (
            SELECT DISTINCT clientrid AS clientid
            FROM [clients]..tblrosters AS r
            WHERE r.status > 0  -- complex criteria 
                AND r.agentrid <> 10
        )
        SELECT  CONVERT(varchar(10),GETDATE(), 120)  AS 'date',
            a.gender, a.cityid , a.city, a.yob,
            a.clientid as clientid, a.serviceid
        FROM Active_accounts_cte a
        -- Count only actors that can be used. Is a member of at least one roster
        LEFT JOIN Roster_cte r ON a.clientid = r.clientid
        WHERE a.clientid IN (SELECT clientid FROM  Roster_cte)
```

### Data wrangler

#### Date and type cast


```sql
select to_char(o.process_date, 'YYYY-MM') as process_date ,
from users.orders o 

select CAST (SUM(Seconds) AS FLOAT)
from time_table
```

#### Table columns and rows manipulations

refs:
* https://modern-sql.com/excel/sumif-in-sql

* Table in the initial format

```sql
select to_char(o.process_date, 'YYYY-MM') as process_date , o.service_category,
		SUM(o.qtd) as qtd
from users.orders o 
group by to_char(o.process_date, 'YYYY-MM'), o.service_category
order by to_char(o.process_date, 'YYYY-MM') desc 
limit 15;
```

process_date|service_category|qtd |
------------|----------------|----|
2020-11     |monthly         | 281|
2020-11     |one_time        |   1|
2020-11     |credit          |   4|
2020-11     |yearly          | 409|
2020-11     |other           |   2|
2020-10     |one_time        |   1|
2020-10     |yearly          |1726|
2020-10     |monthly         | 427|
2020-10     |credit          |   3|
2020-09     |yearly          |1391|
2020-09     |monthly         | 380|
2020-09     |other           |   2|


* Convert to 

```sql
select to_char(o.process_date, 'YYYY-MM') as process_date ,
		SUM(CASE WHEN o.service_category = 'yearly' THEN o.qtd  end) as yearly,
		SUM(CASE WHEN o.service_category = 'monthly' THEN o.qtd  end) as monthly
from users.orders o 
group by to_char(o.process_date, 'YYYY-MM')
order by to_char(o.process_date, 'YYYY-MM') desc 
limit 15;

```


process_date|yearly|monthly|
------------|------|-------|
2020-11     |   409|    281|
2020-10     |  1726|    427|
2020-09     |  1391|    380|
2020-08     |  4559|    260|
2020-07     |   395|    156|
2020-06     |   305|    158|
2020-05     |   154|     75|
2020-04     |    91|     52|
2020-03     |  1427|    134|






```sql
select "date",
	case
		when user_type = 8 then 'Actor'
		when user_type = 2 then 'Director'
		when user_type = 3 then 'Agent'
		else ''
	end as user_type,
from users 
limit 11;
```

### Time Series Analysis time series db


#### Window function: lag and first_value

```sql

-- Create 
CREATE TABLE timestamps (
  ts timestamp,
  n_audios INT
);

INSERT INTO timestamps VALUES
  ('2015-05-01 12:15:23.0', 1),
  ('2015-05-01 12:15:24.0',2),
  ('2015-05-01 12:15:27.0',3),
  ('2015-05-01 12:15:31.0',4),
  ('2015-05-01 12:15:40.0',5),
  ('2015-05-01 12:15:55.0',7),
  ('2015-05-01 12:16:01.0',9),
  ('2015-05-01 12:16:03.0',13),
  ('2015-05-01 12:16:04.0',17),
  ('2015-05-01 12:16:04.0',10);
```


```sql
SELECT
  ts, 
  ts - lag(ts, 1) OVER (ORDER BY ts) delta,
  n_audios,
  n_audios - lag(n_audios , 1) OVER (ORDER BY ts) d_audio
FROM timestamps
ORDER BY ts;

SELECT
  ts, 
  ts - lag(ts, 1) OVER w delta,
  ts - first_value(ts) OVER w total,
  n_audios,
  n_audios - lag(n_audios , 1) OVER w d_audio
FROM timestamps
WINDOW w AS (ORDER BY ts)
ORDER BY ts;

```

#### Window function: move average



refs:
* https://www.compose.com/articles/metrics-maven-calculating-a-moving-average-in-postgresql/

### Summarize data

5 nuymbers: min, mean, median, max and std

refs:
* https://leafo.net/guides/postgresql-calculating-percentile.html


### Take random sample of a table

```sql
with rd_cte 
as
(
    select t.* , random() as rd
    from timestamps t
)
select *
from rd_cte 
where rd > 0.5;


select t.* 
from timestamps t
where random() > 0.5;

-- Create an user defined function
CREATE OR REPLACE FUNCTION random_between(low INT ,high INT) 
   RETURNS INT AS
$$
BEGIN
   RETURN floor(random()* (high-low + 1) + low);
END;
$$ language 'plpgsql' STRICT;

select t.* , random_between(1,100) as rd
from timestamps t;

```

### Materialzed View 

refs:
* https://www.compose.com/articles/its-a-view-its-a-table-no-its-a-materialized-view/